# 🤖 NerdBud – Rule-Based AI Decision Engine

This notebook applies explainable rules to learner performance data
to determine learning readiness and personalized feedback.


In [1]:
import pandas as pd
import os


In [2]:
BASE_PATH = os.path.abspath(os.path.join(os.getcwd(), ".."))
PROFILE_PATH = os.path.join(BASE_PATH, "data", "datasets", "learner_profile.csv")

if not os.path.exists(PROFILE_PATH):
    raise FileNotFoundError("Learner profile not found. Run ProgressTracking first.")

profile = pd.read_csv(PROFILE_PATH)
profile


,topic,accuracy,avg_time,attempts
0,loops,1.0,17.09,1
1,variables,1.0,14.49,1


In [3]:
ADVANCE_ACCURACY = 0.8
SLOW_TIME = 25   # seconds (tunable)


In [4]:
def evaluate_topic(row):
    if row["accuracy"] >= ADVANCE_ACCURACY and row["avg_time"] <= SLOW_TIME:
        return "Strong"
    elif row["accuracy"] >= 0.6:
        return "Moderate"
    else:
        return "Weak"

profile["status"] = profile.apply(evaluate_topic, axis=1)
profile


,topic,accuracy,avg_time,attempts,status
0,loops,1.0,17.09,1,Strong
1,variables,1.0,14.49,1,Strong


In [5]:
def overall_decision(profile_df):
    if (profile_df["status"] == "Weak").any():
        return "Revise"
    elif (profile_df["status"] == "Moderate").any():
        return "Practice"
    else:
        return "Advance"

final_decision = overall_decision(profile)
final_decision


'Advance'

In [6]:
def generate_feedback(profile_df, decision):
    feedback = []

    if decision == "Advance":
        feedback.append("You are performing consistently well. You may proceed to the next level.")

    if decision in ["Practice", "Revise"]:
        weak_topics = profile_df[profile_df["status"] != "Strong"]["topic"].tolist()
        feedback.append(
            f"Focus on improving the following topics: {', '.join(weak_topics)}."
        )

    return feedback

feedback = generate_feedback(profile, final_decision)
feedback


['You are performing consistently well. You may proceed to the next level.']

In [7]:
DECISION_PATH = os.path.join(BASE_PATH, "data", "datasets", "ai_decision.csv")

decision_df = pd.DataFrame([{
    "decision": final_decision,
    "timestamp": pd.Timestamp.now()
}])

decision_df.to_csv(
    DECISION_PATH,
    mode="a",
    header=not os.path.exists(DECISION_PATH),
    index=False
)

DECISION_PATH


'C:\\Users\\Shashank M N\\Desktop\\nerdbud\\data\\datasets\\ai_decision.csv'